This tutorial describes how to evaluate new rules applicable to the assets and derivatives data.

In [1]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst, ViewFileRenderedGrid
from arelle import RenderingEvaluator 

In [2]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import join, isfile
import re
from src import Evaluator
import logging
import data_patterns
import pickle

In [3]:
DECIMALS = 0
RULES_PATH = join('..', 'ftk-rules')
INSTANCES_DATA_PATH = join('..','data','instances','...') #path of folder with converted xbrl-instance data
TEST_DATA_PATH = join('..', 'tests', 'data', 'demo')
RESULTS_PATH = join('..', 'results')
DATA_PATH = join('..', 'data')
logging.basicConfig(filename = join(RESULTS_PATH, 'rules.log'),level = logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Import rules

We start with importing the new rules applicable to the assets and derivatives data. There are several sets of rules applicable to different templates:
* K208A (Information on positions held)
* K208B (Information on assets)
* K208A (Information on positions held) and K208B (Information on assets)
* K210A (Information on positions held) and K210B (Information on derivatives)
* K210B (Information on derivatives)

#### K208

In [4]:
dfr_208 = pd.read_excel(join(RULES_PATH,'FTK_K208.xlsx'))
dfr_208B = pd.read_excel(join(RULES_PATH,'FTK_K208B.xlsx'))
dfr_208A = pd.read_excel(join(RULES_PATH,'FTK_K208A.xlsx'))

#### K210

In [5]:
dfr_210 = pd.read_excel(join(RULES_PATH,'FTK_K210.xlsx'))
dfr_210B = pd.read_excel(join(RULES_PATH,'FTK_K210B.xlsx'))

### Import templates

Next we import the reporting data. In the tutorial 'Convert XBRL-instances to CSV, HTML and pickles' the XBRL-instances are converted to pickle files per template. The pickle files are written to the data/instances folder. We import these pickle files. We merge dataframes for the sets of rules that are applicable to two templates.

#### S06

Import data and make index unique if necessary

In [6]:
# df_208A = pd.read_pickle(join(INSTANCES_DATA_PATH,'FTK.K208A.pickle')).reset_index()
# df_208A['K208A,020A'] = df_208A['K208A,020']
# listt=list(df_208A['K208A,020A'])
# for i in listt:
#     lenn = len(df_208A[df_208A['K208A,020A']==i])
#     if lenn > 1:
#         list_ind = list(df_208A.loc[df_208A['K208A,020A']==i].index)
#         temp = 0
#         for j in list_ind[1:]:
#             temp=temp+1
#             df_208A['K208A,020A'].iloc[j] = df_208A['K208A,020A'].iloc[j] + '_' + str(temp)
# df_208A = df_208A.set_index(['entity', 'period', 'K208A,020A'])

In [7]:
# df_208B = pd.read_pickle(join(INSTANCES_DATA_PATH, 'FTK.K208B.pickle')).reset_index()
# df_208B = df_208B.set_index(['entity', 'period', 'K208B,130'])
# df_208B['K208B,130'] = df_208B.index.get_level_values(2)

In [8]:
# df_208 = pd.merge(pd.read_pickle(join(INSTANCES_DATA_PATH,'FTK.K208A.pickle')).reset_index(),pd.read_pickle(join(INSTANCES_DATA_PATH,'FTK.K208B.pickle')).reset_index(),how='inner', left_on=['entity','period','K208A,020'], right_on=['entity','period','K208B,130']).set_index(['entity', 'period', 'K208A,020'])
# df_208 = df_208.reset_index()
# df_208['K208A,020A'] = df_208['K208A,020']
# listt=list(df_208['K208A,020A'])
# for i in listt:
#     lenn = len(df_208[df_208['K208A,020A']==i])
#     if lenn > 1:
#         list_ind = list(df_208.loc[df_208['K208A,020A']==i].index)
#         temp = 0
#         for j in list_ind[1:]:
#             temp=temp+1
#             df_208['K208A,020A'].iloc[j] = df_208['K208A,020A'].iloc[j] + '_' + str(temp)
# df_208 = df_208.set_index(['entity', 'period', 'K208A,020A'])

In this tutorial we work with dummy data in order to show results

In [9]:
df_208A = pd.read_pickle(join(TEST_DATA_PATH,'FTK.K208A.pickle')).reset_index() #Import demo pickles
df_208B = pd.read_pickle(join(TEST_DATA_PATH,'FTK.K208B.pickle')).reset_index() #Import demo pickles
df_208 = pd.merge(df_208A,df_208B,how='inner', left_on=['entity', 'period', 'K208A,020'], right_on=['entity','period', 'K208B,130']).set_index(['entity', 'period', 'K208A,020'])
df_208B = df_208B.set_index(['entity','period', 'K208B,130'])
df_208A = df_208A.set_index(['entity', 'period', 'K208A,020'])
df_208B['K208B,130'] = df_208B.index.get_level_values(2)
df_208['K208A,020'] = df_208.index.get_level_values(2)

#### S08

In [10]:
# df_210B = pd.read_pickle(join(INSTANCES_DATA_PATH, 'FTK.K210B.pickle')).reset_index()
# df_210B = df_210B.set_index(['entity', 'period', 'K210B,200'])
# df_210B['K210B,200'] = df_210B.index.get_level_values(2)

In [11]:
# df_210 = pd.merge(pd.read_pickle(join(INSTANCES_DATA_PATH,'FTK.K210A.pickle')).reset_index(),pd.read_pickle(join(INSTANCES_DATA_PATH,'FTK.K210B.pickle')).reset_index(),how='inner', left_on=['entity','period','K210A,020'], right_on=['entity','period','K210B,200']).set_index(['entity', 'period', 'K210A,020'])
# df_210 = df_210.reset_index()
# df_210['K210A,020A'] = df_210['K210A,020']
# listt=list(df_210['K210A,020A'])
# for i in listt:
#     lenn = len(df_210[df_210['K210A,020A']==i])
#     if lenn > 1:
#         list_ind = list(df_210.loc[df_210['K210A,020A']==i].index)
#         temp = 0
#         for j in list_ind[1:]:
#             temp=temp+1
#             df_210['K210A,020A'].iloc[j] = df_210['K210A,020A'].iloc[j] + '_' + str(temp)
# df_210 = df_210.set_index(['entity', 'period', 'K210A,020A'])

In this tutorial we work with dummy data in order to show results

In [12]:
df_210A = pd.read_pickle(join(TEST_DATA_PATH,'FTK.K210A.pickle')).reset_index() #Import demo pickles
df_210B = pd.read_pickle(join(TEST_DATA_PATH, 'FTK.K210B.pickle')).reset_index() #Import demo pickles
df_210 = pd.merge(df_210A,df_210B,how='inner', left_on=['entity', 'period', 'K210A,020'], right_on=['entity', 'period', 'K210B,200']).set_index(['entity', 'period', 'K210A,020'])
df_210A = df_210A.set_index(['entity', 'period', 'K210A,020'])
df_210B = df_210B.set_index(['entity', 'period', 'K210B,200'])
df_210B['K210B,200'] = df_210B.index.get_level_values(2)
df_210['K210A,020'] = df_210.index.get_level_values(2)

### Evaluate rules

Now we are ready to evaluate the different sets of rules. First, we construct a PatternMiner-object with the data-patterns package using the rules dataframe. Second, we use the analyze-function to get the results of the rules. We do this for each set of rules separately.

#### S06


In [13]:
miner = data_patterns.PatternMiner(df_patterns=dfr_208)
results_208 = miner.analyze(df_208)
results_208

100%|█████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:09<00:00, 150.24it/s]


result_type        pattern_id  cluster  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2        False  Total solvency 1        0   
                                Key 2        False  Total solvency 2        0   

                                       support  exceptions  confidence  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2        0           1         0.0   
                                Key 2        0           1         0.0   

                                                                             pattern_def  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2  IF ({"FTK.K208A,C070"} != 0) THEN (ABS({"FTK.K...   
                                Key 2  IF ({"FTK.K208A,C060"} != 0) THEN (ABS({"FTK.K...   

                                         P values  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2  1038000.00   
                                Key 2        0.24   

                                                                                Q values  
LEGALIDENTIFIER20P32 2018-12-31 Key 2  [1041000.0, 1038000.0, 0.0315, 1040000.0, 1041...  
                                Key 2  [1041000.0, 0.24, 1042000.0, 1040000.0, 104100...

In [14]:
miner = data_patterns.PatternMiner(df_patterns=dfr_208B)
results_208B = miner.analyze(df_208B)
results_208B

100%|███████████████████████████████████████████████████████████████████████████| 11991/11991 [01:33<00:00, 127.93it/s]


result_type       pattern_id  cluster  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2        False  Unit Solvency 1        0   
                                Key 2        False  Unit Solvency 2        0   

                                       support  exceptions  confidence  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2        0           1         0.0   
                                Key 2        0           1         0.0   

                                                                             pattern_def  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2  IF ({"FTK.K208B,C280"} != 0) THEN ({"FTK.K208B...   
                                Key 2  IF ({"FTK.K208B,C290"} != 0) THEN ({"FTK.K208B...   

                                           P values      Q values  
LEGALIDENTIFIER20P32 2018-12-31 Key 2  1.042000e+06  3.150000e-02  
                                Key 2  3.150000e-02  1.042000e+06

In [15]:
miner = data_patterns.PatternMiner(df_patterns=dfr_208A)
results_208A = miner.analyze(df_208A)
results_208A

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 111.19it/s]


result_type pattern_id  cluster  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2        False        Par        0   
                                Key 2        False   Quantity        0   

                                       support  exceptions  confidence  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2        0           1         0.0   
                                Key 2        0           1         0.0   

                                                                             pattern_def  \
LEGALIDENTIFIER20P32 2018-12-31 Key 2  IF ({"FTK.K208A,C060"} != 0) THEN ({"FTK.K208A...   
                                Key 2  IF ({"FTK.K208A,C070"} != 0) THEN ({"FTK.K208A...   

                                         P values    Q values  
LEGALIDENTIFIER20P32 2018-12-31 Key 2        0.24  1038000.00  
                                Key 2  1038000.00        0.24

#### S08

In [16]:
miner = data_patterns.PatternMiner(df_patterns=dfr_210)
results_210 = miner.analyze(df_210)
results_210

100%|███████████████████████████████████████████████████████████████████████████████| 126/126 [00:00<00:00, 151.08it/s]


result_type       pattern_id  cluster  \
LEGALIDENTIFIER20P32 2018-12-31 Key 9         True  Contract size 2        0   
                                Key 9         True   Buyer/seller 2        0   

                                       support  exceptions  confidence  \
LEGALIDENTIFIER20P32 2018-12-31 Key 9        1           0         1.0   
                                Key 9        1           0         1.0   

                                                                             pattern_def  \
LEGALIDENTIFIER20P32 2018-12-31 Key 9  IF {"FTK.K210B,C290"} = "USA2" THEN {"FTK.K210...   
                                Key 9  IF {"FTK.K210B,C290"} = "USA2" THEN {"FTK.K210...   

                                      P values   Q values  
LEGALIDENTIFIER20P32 2018-12-31 Key 9     USA2  1.051e+06  
                                Key 9     USA2       Open

In [17]:
miner2 = data_patterns.PatternMiner(df_patterns=dfr_210B)
results2_210B = miner2.analyze(df_210B)
results2_210B

100%|███████████████████████████████████████████████████████████████████████████████| 568/568 [00:04<00:00, 114.00it/s]


result_type             pattern_id  \
LEGALIDENTIFIER20P32 2018-12-31 Key 9         True             Currency 1   
                                Key 9         True  Credit quality step 2   

                                       cluster  support  exceptions  \
LEGALIDENTIFIER20P32 2018-12-31 Key 9        0        1           0   
                                Key 9        0        1           0   

                                       confidence  \
LEGALIDENTIFIER20P32 2018-12-31 Key 9         1.0   
                                Key 9         1.0   

                                                                             pattern_def  \
LEGALIDENTIFIER20P32 2018-12-31 Key 9  IF ({"FTK.K210B,C290"} = "USA2") THEN (({"FTK....   
                                Key 9  IF (({"FTK.K210B,C230"}= "BBB+") & ({"FTK.K210...   

                                            P values               Q values  
LEGALIDENTIFIER20P32 2018-12-31 Key 9           USA2            [0, 0, EUR]  
                                Key 9  [BBB+, Fitch]  Credit quality step 3